<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Compare_Jacobian_Diag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Compare Diagonal Jacobian on ResNet 1 batch

Why is my data different from Johns ?


In [2]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [0]:
thismodel = 'resnet20_cifar10'
inum_start = 0

In [4]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=f89b63f6bc159fb601861d8362f92d1ca970d1405b720f25d4f61ac34ca41ab2
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [5]:
!pip install pytorchcv

     |████████████████████████████████| 440kB 4.8MB/s 
  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24787 sha256=145d1f884a4ec977e2534b9f726fe9baae4e3ac5efcd9fee398807c41d165b01
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw
time: 6.66 s


In [6]:
!pip install GPUtil
import GPUtil

GPUtil.showUtilization()

  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=ea3636c6830aa36e86424231b9c042d0553bf60b9dc42b18f2a4574160c6f2cf
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
time: 3.52 s


In [7]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy


/bin/bash: import: command not found
time: 1.59 s


In [8]:
def get_data(batch_size=100, train_range=None, test_range=None):
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)  
  
  if train_range:
      train_dataset = torch.utils.data.Subset(train_dataset, train_range)

  if test_range:
      teat_dataset = torch.utils.data.Subset(test_dataset, test_range)


  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size,
                          num_workers=4,
                          shuffle=False)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         num_workers=4,
                         shuffle=False)
  return train_dataset, test_dataset, train_loader, test_loader

time: 12.4 ms


### $J$ Jacobian (replaces native pytorch 1.5 code)

Uses autograd 





In [42]:
from torch.autograd.gradcheck import zero_gradients

def fast_jacobian(inputs, output, batch_size, data_dim):
	"""
	input: input for the function for which the Jacobian will
	computed. It will be batch_size*data_dim. Make sure that the
	input is flagged as requires_grad=True with the torch.autograd.Variable
	wrapper. 
	output: output of the function for which the Jacobian will
	be computed. It will be batch_size*classes
	return: Jacobian of dimension batch_size*classes*data_dim
	"""
	assert inputs.requires_grad
	print("inputs: ",inputs.shape)
	num_classes = output.size()[1] #0 index is batch

	jacobian = torch.zeros(num_classes, *inputs.size())
	grad_output = torch.zeros(*output.size())
	if inputs.is_cuda:
		grad_output = grad_output.cuda()
		jacobian = jacobian.cuda()

	#zero out gradients
	#compute gradient for one of the classes
	for i in range(num_classes):
		zero_gradients(inputs)
		grad_output.zero_()
		grad_output[:,i] = 1
		output.backward(grad_output, retain_graph=True)
		jacobian[i] = inputs.grad.data

	J =  torch.transpose(jacobian, dim0=0, dim1=1)
	print("fast J: ",J.shape, data_dim)
	J = J.reshape(batch_size, data_dim)
 
	return J


time: 11.2 ms


### Compare old and new diagonal;


### ResNet 

In [10]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
        print(modelname)

resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
time: 98.4 ms


In [44]:
num_classes= 10
device = 'cuda:0'

data_dim = 3*32*32

model = ptcv_get_model(thismodel, pretrained=True)
model = model.to(device)

for batch_size in range(1,5):
  print("batch size ", batch_size)
  train_dataset, test_dataset, train_loader, test_loader = get_data(batch_size=batch_size)

  for batch, data in enumerate(test_loader):
    features, labels = data

    inputs = features.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)

    Jnew = fast_jacobian(inputs, outputs, batch_size, num_classes*data_dim)
    j0new = Jnew.cpu().numpy()[0,:]

    Jold = torch.autograd.functional.jacobian(model, inputs)
    print(Jold.shape)
    j0orig = Jold.cpu().numpy()[0,:,0,:,:,:]
    
    Jold = Jold.view(batch_size,num_classes*batch_size*data_dim)
    j0old = Jold.cpu().numpy()[0,:]

    print("||jnew|| ", np.linalg.norm(j0new))
    print("||jorig|| ", np.linalg.norm(j0orig))
    print("||jold|| ", np.linalg.norm(j0old))
    print(".   ")

    break



batch size  1
Files already downloaded and verified
Files already downloaded and verified
inputs:  torch.Size([1, 3, 32, 32])
fast J:  torch.Size([1, 10, 3, 32, 32]) 30720
torch.Size([1, 10, 1, 3, 32, 32])
||jnew||  11.510124
||jorig||  11.510125
||jold||  11.510125
.   
batch size  2
Files already downloaded and verified
Files already downloaded and verified
inputs:  torch.Size([2, 3, 32, 32])
fast J:  torch.Size([2, 10, 3, 32, 32]) 30720
torch.Size([2, 10, 2, 3, 32, 32])
||jnew||  5.987037
||jorig||  12.755404
||jold||  18.5789
.   
batch size  3
Files already downloaded and verified
Files already downloaded and verified
inputs:  torch.Size([3, 3, 32, 32])
fast J:  torch.Size([3, 10, 3, 32, 32]) 30720
torch.Size([3, 10, 3, 3, 32, 32])
||jnew||  6.950019
||jorig||  19.470583
||jold||  27.521496
.   
batch size  4
Files already downloaded and verified
Files already downloaded and verified
inputs:  torch.Size([4, 3, 32, 32])
fast J:  torch.Size([4, 10, 3, 32, 32]) 30720
torch.Size([4, 1